# ***Accessing Google Drive***

In [1]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [2]:
import pandas as pd
import io
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
import hashlib 

In [3]:
!pip install jupyter_plotly_dash


     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 1.3MB 8.8MB/s 
     |████████████████████████████████| 512kB 31.6MB/s 
     |████████████████████████████████| 194kB 33.3MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 1.0MB 42.9MB/s 
     |████████████████████████████████| 3.5MB 37.1MB/s 
     |████████████████████████████████| 296kB 43.9MB/s 
     |████████████████████████████████| 143kB 49.2MB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
     |████████████████████████████████| 1.8MB 35.7MB/s 
     |████████████████████████████████| 358kB 49.7MB/s 
  Created wheel for dpd-components: filename=dpd_components-0.1.0-cp36-none-any.whl size=4568 sha256=d6e087cbdfa88759ed44830ba759921285dea57b9c4714af1523cf2db008b78d
  Stored in directory: /root/.cache/pip/wheels/36/93/4f/690abc41d258cac9a7d6246873294c1e11300dc74357d6d146
  Created wheel for dash-html-components: filename=dash_html

# ***Helper Function***

In [4]:

def count_All_Pictures(folder_id,path):
  
  global number_Of_Text_File
  global number_Of_Picture
  global video_Time
  global id_Not_In
  global count
  page_token_parent = None

  check_duplicates = []

  while True:
    response_Parent = drive_service.files().list(q = "'"+folder_id+"' in parents and trashed = false",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name,videoMediaMetadata)',
                                          pageToken=page_token_parent).execute()

    for files in response_Parent.get('files',[]):
                
                file_Or_Folder = files.get('name').split('.')
                
                if(file_Or_Folder[0] != 'picture_contaminated' and file_Or_Folder[0] != 'video_contaminated'):

                  if( len(file_Or_Folder) == 2 and files.get('name') not in check_duplicates ):
                    
                    check_duplicates.append(files.get('name'))
                    if(file_Or_Folder[1] == 'jpg' ):  
                      number_Of_Picture = number_Of_Picture + 1
                    elif(file_Or_Folder[1] == 'txt' and "video" not in path):
                      number_Of_Text_File = number_Of_Text_File + 1
                    elif(file_Or_Folder[1] == 'mp4'):
                      if(files.get('videoMediaMetadata') == None):
                        video_Time = video_Time + 0
                      else:
                        video_Time = video_Time + (float(files.get('videoMediaMetadata')['durationMillis'])/1000)

                  elif( len(file_Or_Folder) != 2 ):
                   
                    path.append(files.get('name'))
                    count_All_Pictures(files.get('id'),path)
                    parent = path.pop(-1)

                    if(parent == 'picture'):
                        if id_Not_In:
                          student_Id.append(path[0])
                          text_File_Generated.append(number_Of_Text_File) 
                          picture_Collected.append(number_Of_Picture)
                          number_Of_Picture = 0
                          number_Of_Text_File = 0
                          id_Not_In = False
                        else:
                          text_File_Generated.append(number_Of_Text_File) 
                          picture_Collected.append(number_Of_Picture)
                          number_Of_Picture = 0
                          number_Of_Text_File = 0               
                    elif(parent == 'video'):

                        if id_Not_In:
                          student_Id.append(path[0])
                          video_Time_Collected.append(video_Time)
                          video_Time = 0
                          id_Not_In = False
                        else:
                          video_Time_Collected.append(video_Time)
                          video_Time = 0
                    if(len(path) == 0):
                          id_Not_In = True
                         


    page_token_parent = response_Parent.get('nextPageToken', None)
    if page_token_parent is None:
          break


# ***Analysis of the collected Data***

In [5]:
student_Id = []
picture_Collected = []
text_File_Generated = []
number_Of_Text_File = 0
number_Of_Picture = 0
video_Time_Collected  = []
video_Time = 0
id_Not_In = True
folder_id = "1N9pulzZSKSgT8SCK_FA2x0jKH_KWmcZ5"
path = []

count_All_Pictures(folder_id,path)

In [6]:
data = {'ID':student_Id, 'Collected_Pics': picture_Collected , 'Text_Generated': text_File_Generated ,'Video_Collected_Time(sec)': video_Time_Collected } 

live_Stats = pd.DataFrame(data)


In [7]:
pd.set_option('display.max_rows', 100)

In [8]:
live_Stats

,ID,Collected_Pics,Text_Generated,Video_Collected_Time(sec)
0,1511198042_Please_Knock_me_Third_Batch,100,100,4770.795
1,1620379042_**_First_Batch,221,221,3760.472
2,1632267642_First_Batch,220,220,3624.363
3,1510295042_First_Batch,223,223,3794.522
4,1510708042_First_Batch,220,220,0.000
5,1511423042_First_Batch,231,226,4436.886
6,1520198642_First_Batch,232,232,2310.588
7,1520469642_First_Batch,226,226,3971.344
8,1610544042_**_First_Batch,154,154,3620.718
9,1611822042_First_Batch,202,202,4382.885


# ***Picture Collected In Map***

In [ ]:

def Generate_Overall_DataFrame(folder_id,path):
  
  check_duplicates = []
  page_token_parent = None
  while True:
    response_Parent = drive_service.files().list(q = "'"+folder_id+"' in parents and trashed = false",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name,videoMediaMetadata)',
                                          pageToken=page_token_parent).execute()

    for files in response_Parent.get('files',[]):
                
                file_Or_Folder = files.get('name').split('.')
                
                if(file_Or_Folder[0] != 'picture_contaminated' and file_Or_Folder[0] != 'video_contaminated'):
                  if(len(file_Or_Folder) == 2 and files.get('name') not in check_duplicates ):
                    check_duplicates.append(files.get('name'))
                    if('picture' in path):
                      if(file_Or_Folder[1] != 'jpg' and file_Or_Folder[1] == 'txt'):
                        request = drive_service.files().get_media(fileId=files.get('id'))
                        fh = io.BytesIO()
                        downloader = MediaIoBaseDownload(fh, request)
                        status, done = downloader.next_chunk()
                        data = fh.getvalue().decode("utf-8").split("\n")
                        if hashlib.md5(data[0].encode('utf-8')).hexdigest() == data[1]:
                          collected_Data = data[0].split(";")
                          ID_pic.append(path[0])
                          file_Name.append(files.get('name').split('_')[0])
                          tamper_Statues.append("Not Tampered")
                          depth.append(collected_Data[0])
                          length.append(collected_Data[1])
                          width.append(collected_Data[2])
                          latitude.append(collected_Data[3])
                          longitude.append(collected_Data[4])
                        else:
                          ID_pic.append(path[0])
                          tamper_Statues.append("Tampered")
                          depth.append(None)
                          length.append(None)
                          width.append(None)
                          latitude.append(None)
                          longitude.append(None)

                    elif("video" in path):
                      if(file_Or_Folder[1] != 'mp4' and file_Or_Folder[1] == 'txt'):
                          ID_video.append(path[0]) 
                          video_File_Id.append(files.get('id'))

                                                

                  elif(len(file_Or_Folder) != 2 ):                   
                    path.append(files.get('name'))
                    Generate_Overall_DataFrame(files.get('id'),path)
                    path.pop(-1)
  

                         


    page_token_parent = response_Parent.get('nextPageToken', None)
    if page_token_parent is None:
          break

In [ ]:
folder_id = "1N9pulzZSKSgT8SCK_FA2x0jKH_KWmcZ5"
path = []
ID_pic = []
tamper_Statues = []
depth = []
length = []
width = [] 
ID_video = []
latitude = []
longitude = []
file_Name = []
video_File_Id = []
path = []


In [ ]:
Generate_Overall_DataFrame(folder_id,path)

In [ ]:
Data_all_pic = {"Student_ID": ID_pic,
            "Depth": depth,
            "Length": length,
            "Width": width,
            "Latitude": latitude,
            "Longitude": longitude,
            "Tamper Status": tamper_Statues,
           
            }
Data_all_video ={
                "Student_ID": ID_video,
                 "Video_File_ID": video_File_Id
                } 
           

In [ ]:
df_pic = pd.DataFrame (Data_all_pic, columns = ['Student_ID',"Depth","Length","Width","Latitude","Longitude","Tamper Status"])
df_vid = pd.DataFrame (Data_all_video, columns = ['Student_ID',"Video_File_ID"])

In [ ]:
df_pic.to_csv('Cached_Data_pic.csv')
df_vid.to_csv('Cached_Data_video.csv')
cached_Array = ['Cached_Data_pic.csv','Cached_Data_video.csv']

In [ ]:

for file_name in cached_Array:
  file_metadata = {'name': file_name,'parents':['17E9jnyv6iaXshHok-TVQdSYvTvauwmCk']}
  media = MediaFileUpload(file_name)
  file = drive_service.files().create(body=file_metadata,
                                      media_body=media,
                                      fields='id').execute()


# ***Upload To Google Drive***